# Preprocess Data

In [1]:
import pandas as pd
import json
from tqdm import tqdm

In [2]:
android_df = pd.read_csv('raw/dataTest_device_profile_android.csv')
ios_df = pd.read_csv('raw/dataTest_device_profile.csv')
joint_df = pd.concat([android_df, ios_df])
joint_df['params'] = joint_df['params'].apply(lambda x: json.loads(x))
joint_df.head()

,timestamp,userId,params
0,1596244856,170425000002238,{'hw_released': 'Tue Jul 07 08:50:39 GMT+07:00...
1,1596244830,191113586003363,{'hw_released': 'Thu Jul 02 12:19:14 GMT+07:00...
2,1596244888,200413000001404,{'hw_released': 'Thu Dec 07 11:24:05 GMT+07:00...
3,1596244831,200428000037320,{'hw_released': 'Wed Apr 22 07:53:33 GMT+07:00...
4,1596244849,190309000002524,{'hw_released': 'Thu Mar 12 12:05:05 GMT+07:00...


In [3]:
formatted = pd.json_normalize(joint_df['params'])

In [4]:
joint_df.reset_index(drop=True, inplace=True)

formatted.reset_index(drop=True, inplace=True)

In [5]:
new_df = pd.concat([joint_df[['timestamp', 'userId']], formatted], axis=1)
new_df.drop_duplicates(inplace=True)
new_df.head()

,timestamp,userId,hw_released,hw_device_id,battery_voltage,hw_board,hw_cpu_name,hw_screen_aspect_ratio,hw_screen_class,battery_type,...,sim_operator_name,hw_camera_front_optical_stabilization,hw_camera_front_resolution,hw_camera_back_resolution,sim_operator,hw_camera_back_longest_exposure_value,hw_camera_back_supported_resolution,hw_capacity_force_touch,os_multitasking,cellular_capabilities
0,1596244856,170425000002238,Tue Jul 07 08:50:39 GMT+07:00 2020,f21ebb2f9064b16e,4091 mV,exynos9810,exynos9810,2,Normal,Li-ion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1596244830,191113586003363,Thu Jul 02 12:19:14 GMT+07:00 2020,13cf342d200936c8,4260 mV,AGS2,hi6250,1,Large,Li-poly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1596244888,200413000001404,Thu Dec 07 11:24:05 GMT+07:00 2017,b41f717314855504,3508 mV,SC9830I,sc8830,1,Normal,Li-ion,...,vn,NO,4 MP,7 MP,45204,,3264x2448\n2592x1936\n2048x1536\n1600x1200\n12...,NaN,NaN,NaN
3,1596244831,200428000037320,Wed Apr 22 07:53:33 GMT+07:00 2020,8d36707898c6955d,3877 mV,exynos9810,samsungexynos9810,2,Normal,Li-ion,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1596244849,190309000002524,Thu Mar 12 12:05:05 GMT+07:00 2020,af1934dafd8aba01,3644 mV,mt6757,mt6757,1,Normal,Li-ion,...,NaN,NO,15 MP,12 MP,NaN,,4128x3096\n4128x2320\n3264x1836\n3264x2448\n30...,NaN,NaN,NaN


In [6]:
required_cols = ['timestamp', 'userId', 'hw_released', 'hw_device_id', 'hw_cpu_name',
       'hw_screen_aspect_ratio', 'hw_screen_pixel_density', 'system_packages',
       'network_wifi_mac_address', 'hw_screen_resolution', 'os_version',
       'hw_device_manufacturer', 'hw_cpu_speed', 'hw_device_model',
       'hw_screen_refresh_rate', 'os_name', 'os_root_access',
       'hw_cpu_supported_64_bit_abis', 'user_agent', 'hw_device_string',
       'os_version_name', 'hw_cpu_core_count', 'hw_cpu_supported_32_bit_abis',
       'hw_cpu_processor', 'hw_cpu_manufacturer', 'hw_screen_size',
       'hw_bluetooth_address', 'installed_packages', 'hw_ram_total',
       'hw_cpu_min_speed', 'hw_storage_total', 'list_of_wifi',
       'hw_camera_front_max_photo_resolution',
       'hw_camera_back_max_video_resolution', 'ringtone',
       'hw_camera_number_camera', 'hw_camera_back_max_photo_resolution',
       'hw_camera_front_max_video_resolution']

In [7]:
df = new_df[required_cols]

In [8]:
df.duplicated().value_counts()

False    495535
True         39
dtype: int64

In [9]:
df = df.drop_duplicates()

# Insert data into ArangoDB

## Add users

In [10]:
unique_users = df['userId'].unique()

In [11]:
user_list = []
for u in tqdm(unique_users):
    user_list.append({
        '_key': str(u.strip())
    })

  0%|          | 0/409836 [00:00<?, ?it/s]


AttributeError: 'numpy.int64' object has no attribute 'strip'

In [ ]:
with open('json/users.json', 'w') as f:
    json.dump(user_list, f)

In [30]:
# Convert json to jsonl
!jq -c ".[]" json/users.json > jsonl/users.jsonl

In [33]:
# import to arangodb
# arangoimport --server.database=final --create-collection true --collection users --type jsonl --file "jsonl/users.jsonl" --progress true --threads 16

## Add latest devices info

In [34]:
devices_latest = df[df.groupby(['hw_device_id'])['timestamp'].transform(max) == df['timestamp']]

In [35]:
def format_packages(x):
    return x.split(', ') if type(x) is not float else 'nan'

In [36]:
def format_list_of_wifi(x):
    if type(x) is not float and x != '':
        return x.replace('"', '').split(', ')
    else:
        return 'nan'

In [37]:
def format_ringtone(x):
    return x.strip(',').split(',') if type(x) is not float else 'nan'

In [41]:
device_info = []
cols_exclude_devices = set(df.columns) - set(['hw_device_id', 'installed_packages', 'system_packages', 'list_of_wifi', 'ringtone'])
with tqdm(total=len(devices_latest)) as pbar:
    for _, row in devices_latest.iterrows():
        e = {
            '_id': row['hw_device_id'],
            '_key': row['hw_device_id'],
            'installed_packages': format_packages(row['installed_packages']),
            'system_packages': format_packages(row['system_packages']),
            'list_of_wifi': format_list_of_wifi(row['list_of_wifi']),
            'ringtone': format_ringtone(row['ringtone'])
        }
        for c in cols_exclude_devices:
            e[c] = str(row[c])
        device_info.append(e)
        pbar.update(1)

100%|██████████| 407741/407741 [01:39<00:00, 4114.40it/s]


In [42]:
with open('json/devices.json', 'w') as f:
    json.dump(device_info, f)

In [43]:
# Convert json to jsonl
!jq -c ".[]" json/devices.json > jsonl/devices.jsonl

In [ ]:
# arangoimport --server.database=final --create-collection true --collection devices --type jsonl --file "jsonl/devices.jsonl" --progress true --threads 16

## Add users - devices 

In [30]:
user_devices = []
cols_exclude_users_devices = set(df.columns) - set(['userId', 'hw_device_id', 'installed_packages', 'system_packages', 'list_of_wifi', 'ringtone'])
with tqdm(total=len(df)) as pbar:
    for _, row in df.iterrows():
        ud = {
            '_key': f"{str(row['userId']).strip()}_{str(row['hw_device_id'])}_{row['timestamp']}",
            '_from': 'users/' + str(row['userId']).strip(),
            '_to': 'devices/' + str(row['hw_device_id']).strip(),
            'type': 'user_use_device'
        }
        for c in cols_exclude_users_devices:
            ud[c] = str(row[c]).strip()
        user_devices.append(ud)
        pbar.update(1)

100%|██████████| 495535/495535 [01:39<00:00, 4980.65it/s]


In [31]:
with open('json/users_use_devices.json', 'w') as f:
    json.dump(user_devices, f)

In [32]:
# Convert json to jsonl
!jq -c ".[]" json/users_use_devices.json > jsonl/users_use_devices.jsonl

In [ ]:
# arangoimport --server.database=final --create-collection-type edge --collection users_devices  --type jsonl --file "jsonl/users_use_devices.jsonl" --progress true --threads 16

## Devices model

In [ ]:
unique_device_model = df['hw_device_model'].unique()
inserted = []
for uq in unique_device_model:
    formatted = '_'.join(x for x in uq.split(' '))
    if formatted not in inserted:
        inserted.append(formatted)
        device_model.createDocument({
            '_key': formatted
        }).save()

## Device - DeviceModel

In [16]:
for _, row in df.iterrows():
    formatted_device_model = '_'.join(x for x in row['hw_device_model'].split(' '))
    new_ddm = device_deviceModel.createDocument({
        '_from': 'devices/' + str(row['hw_device_id']),
        '_to': 'device_model/' + formatted_device_model,
        'timestamp': str(row['timestamp']),
        'userId': str(row['userId'])
    })
    new_ddm.save()

# Test

In [10]:
df.head()

,timestamp,userId,hw_released,hw_device_id,hw_cpu_name,hw_screen_aspect_ratio,hw_screen_pixel_density,system_packages,network_wifi_mac_address,hw_screen_resolution,...,hw_ram_total,hw_cpu_min_speed,hw_storage_total,list_of_wifi,hw_camera_front_max_photo_resolution,hw_camera_back_max_video_resolution,ringtone,hw_camera_number_camera,hw_camera_back_max_photo_resolution,hw_camera_front_max_video_resolution
0,1596244856,170425000002238,Tue Jul 07 08:50:39 GMT+07:00 2020,f21ebb2f9064b16e,exynos9810,2,420 Dpi,"com.samsung.android.provider.filterprovider, c...",12:2F:14:33:9D:E7,1080x2400,...,"7,94 GB",455000Hz,118 GB,,NaN,NaN,NaN,NaN,NaN,NaN
1,1596244830,191113586003363,Thu Jul 02 12:19:14 GMT+07:00 2020,13cf342d200936c8,hi6250,1,320 Dpi,"com.huawei.hifolder, com.android.cts.priv.ctss...",A8:E5:44:0E:BD:BB,1200x1920,...,"2,88 GB",480000Hz,"25,23 GB","""Dinh1"", ""MaiCa"", ""NhuQuynh"", ""Support"", ""Nam ...",NaN,NaN,NaN,NaN,NaN,NaN
2,1596244888,200413000001404,Thu Dec 07 11:24:05 GMT+07:00 2017,b41f717314855504,sc8830,1,320 Dpi,"com.monotype.android.font.rosemary, com.sec.an...",AC:AF:B9:61:36:C8,720x1280,...,"1,33 GB",768000Hz,"4,79 GB","""Xiaomi_70BE_8F41"", ""COM TAM SONG ANH"", ""Redmi...",2576x1932,3264x2448,"Basic Bell,Basic Tone,Beep Once,Beep-Beep,Begi...",2,3264x2448,2576x1932
3,1596244831,200428000037320,Wed Apr 22 07:53:33 GMT+07:00 2020,8d36707898c6955d,samsungexynos9810,2,420 Dpi,"com.samsung.android.provider.filterprovider, c...",24:18:1D:FE:E3:E0,1080x2220,...,"5,89 GB",455000Hz,120 GB,,NaN,NaN,NaN,NaN,NaN,NaN
4,1596244849,190309000002524,Thu Mar 12 12:05:05 GMT+07:00 2020,af1934dafd8aba01,mt6757,1,420 Dpi,"com.samsung.android.provider.filterprovider, c...",A4:6C:F1:C3:34:4E,1080x1920,...,3.92 GB,247000Hz,26.75 GB,"""NamCoffee&Milktea"", ""YÊU MỘT NGƯỜI"", ""Studio1...",4608x3456,4128x3096,NaN,2,4128x3096,4608x3456


In [15]:
df.groupby(['userId']).agg({'hw_device_id': 'count'}).sort_values(by="hw_device_id")

,hw_device_id
userId,
160514000003501,1
200124000031214,1
200124000031205,1
200124000031090,1
200124000031055,1
...,...
191101679002231,67
171106000000266,68
190224000013026,86


In [12]:
df.groupby('hw_device_id').agg({'userId': lambda x: len(set(x))}).sort_values('userId', ascending=False)[:10]

,userId
hw_device_id,
460968F8-068F-41C8-B130-2F5F7E968C9C,49
A5AA40DE-35CB-4406-B4E9-B10E497766A0,35
EDF10704-E7E4-4CC6-BA25-9A30C7720D02,32
027894A1-E617-4A74-9035-33E36FE16800,32
0CA30A94-6251-4727-8340-9B6BE942AACB,31
F8E47BD8-EA84-4621-94C8-104B535C3097,21
51E7555D-BC41-483A-B627-7FD3A6693E4E,20
3C9D6E46-A3C6-456E-BB39-48C0154C7F5A,20
3A8BA073-A02B-47E1-81EF-C69A62A879C2,19


In [ ]:
df.groupby('hw_device_id').agg({'userId': lambda x: len(set(x))}).sort_values('userId')